In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16

In [2]:
b_size = 5

train_datagen = ImageDataGenerator(rescale=1./255,
                                    horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  fill_mode='nearest')

train_generator = train_datagen.flow_from_directory('train', target_size=(150, 150),
                                                  batch_size=b_size,
                                                  class_mode='binary')

Found 160 images belonging to 2 classes.


In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('test', target_size=(150, 150),
                                                  batch_size=b_size,
                                                  class_mode='binary')

Found 120 images belonging to 2 classes.


In [4]:
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
transfer_model.trainable = False
transfer_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [5]:
finetune_model = Sequential()
finetune_model.add(transfer_model)
finetune_model.add(Flatten())
finetune_model.add(Dense(64, activation='relu'))
finetune_model.add(Dense(2, activation='softmax'))
finetune_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                524352    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 130       
Total params: 15,239,170
Trainable params: 524,482
Non-trainable params: 14,714,688
_________________________________________________________________


In [6]:
finetune_model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=optimizers.Adam(learning_rate=0.0002),
             metrics=['accuracy'])

steps_train = len(train_generator)
steps_test = len(test_generator)

hist = finetune_model.fit(train_generator, steps_per_epoch=steps_train,
                epochs=20, validation_data=test_generator,
                validation_steps=steps_test)

Train for 32 steps, validate for 24 steps
Epoch 1/20
32/32 [==============================] - 40s 1s/step - loss: 0.6697 - accuracy: 0.6062 - val_loss: 0.5085 - val_accuracy: 0.8583
Epoch 2/20
32/32 [==============================] - 38s 1s/step - loss: 0.4526 - accuracy: 0.8687 - val_loss: 0.4023 - val_accuracy: 0.8667
Epoch 3/20
32/32 [==============================] - 41s 1s/step - loss: 0.3476 - accuracy: 0.9125 - val_loss: 0.3200 - val_accuracy: 0.9000
Epoch 4/20
32/32 [==============================] - 41s 1s/step - loss: 0.2842 - accuracy: 0.9062 - val_loss: 0.2731 - val_accuracy: 0.9167
Epoch 5/20
32/32 [==============================] - 42s 1s/step - loss: 0.2570 - accuracy: 0.9187 - val_loss: 0.2817 - val_accuracy: 0.8750
Epoch 6/20
32/32 [==============================] - 43s 1s/step - loss: 0.2027 - accuracy: 0.9312 - val_loss: 0.2109 - val_accuracy: 0.9167
Epoch 7/20
32/32 [==============================] - 42s 1s/step - loss: 0.1673 - accuracy: 0.9563 - val_loss: 0.2034 -